# Using rapidfuzz to extract affiliatuons from RES

First the JSON output file from tezseract is read in and made into a useable dataframe

In [1]:
#Importing the libraries
import json
import pandas as pd
from rapidfuzz import process
import requests

# Load input files from github
#tesseract_input = requests.get('https://raw.githubusercontent.com/Matt970204/Affiliations_V2/main/1_Input%20files%20and%20processing%20input%20files/1_1_Input%20files/Tesseract%20affiliation%20output/AER/AER_aff_output_1971_2010.json')
#json_data = tesseract_input.json()

# Import JSON file
f = open('/Users/MattJohnson/Desktop/Affiliations/1_Input files and processing input files/1_1_Input files/Tesseract affiliation output/JPE/JPE_affiliation_output_1940_2010.json')
json_data = json.load(f)

# Define Journal and Dataframe
Journal_Name = 'JPE'
Reference_Data_List = []

# Extract Json Data into dataframe
for key1, value1 in json_data.items():
    Article_ID = key1
    content_type = value1["content_type"]
    author_list = value1["authors"]
    authorsNo = len(author_list)
    stable_url = value1["stable_url"]
    for key2 in value1["affiliations"]:
        Ref_type = key2
        if key2 == "found":
            ref_string = value1["affiliations"][key2]
        else: 
            for key3 in value1["affiliations"]["raw"]:
                if len(value1["affiliations"]["raw"]) > 1:
                    print(key1, "   ",len(value1["affiliations"]["raw"]))
                if key3 == "1":
                    print(key3)
                    ref_string = value1["affiliations"][key2][key3]
                    print(ref_string)

    Reference_Data_List.append({'Journal_Name' : Journal_Name,
    'Authors_count' : authorsNo, 
    'Authors_list' : author_list,
    'Article_ID' : Article_ID, 'content_type' : content_type,
    'stable_url' : stable_url, 
    'Ref_type': Ref_type,  'ref_string':ref_string })


JPE_aff_ref_output_1940_2010 = pd.DataFrame.from_records(Reference_Data_List)     
JPE_aff_ref_output_1940_2010.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/Working files/JPE_aff_ref_output_1940_2010.xlsx')        

1
['Preface\n', 'The papers in this issue of the /PE were presented at a Festschrift con-\nference in honor of James J. Heckman\'s 65th birthday. The conference\nattendees included many of his former graduate students, coauthors,\nand colleagues. Thirteen excellent papers were submitted to this special\nissue. They went through the regular refereeing process, and six were\nultimately accepted. Below, we describe how the papers included in this\nissue were inspired and influenced by Heckman\'s research.\n\nHeckman has produced a large body of research examining the ori-\ngins of inequality in socioeconomic outcomes. An important insight\nfrom this research is that the family has a large influence on an individ-\nual\'s human capital formation. This finding led Heckman to state that\n*the accident of birth is a major source of inequality" (2008, 289).! After\nHeckman won the Nobel Prize in 2000, a large part of his research focused\non understanding what types of policy interventions can

Next the Unique list of universities is read in, this is used as a database to check the affiliton string against using rapidfuzz. 

In [2]:
# Load in the list of unique Universites from github stored in 2_fuzzy matching
#affs = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/main/2_Extracting%20affilitations/2_Fuzzy%20matching/University%20list/Unique%20Universities.xlsx")


affs = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/Unique Universities.xlsx')
print(affs.shape)

for i in range(len(affs)):
    #Replaceing title case and stripping leading space
    #affs.iat[i,0] = affs.iat[i,0].strip().title()
    affs.iloc[i]['Affilliation'] = affs.iloc[i]['Affilliation'].strip().title()
    #affs.iat[i,0] = affs.iat[i,0].strip().upper()

Affiliation_choices = affs['Affilliation'].values.tolist()

print(affs['Affilliation'].head())

(850, 14)


/var/folders/23/ft9qf9c11rj_vqhbjml910f40000gn/T/ipykernel_16153/1542333116.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affs.iloc[i]['Affilliation'] = affs.iloc[i]['Affilliation'].strip().title()


0      Yale University
1            Cambridge
2              Harvard
3               Oxford
4    Oxford University
Name: Affilliation, dtype: object


In [3]:
#testdata = RES_1940_2010.loc[0:20].reset_index(drop = True)
testdata = JPE_aff_ref_output_1940_2010

for i in range(len(testdata)):
    if testdata.iloc[i]['Ref_type'] == "raw":
        testdata.loc[i,['ref_string_clean1']] =  " ".join(testdata.iloc[i]['ref_string'])
        #x = " ".join(testdata.iloc[i]['ref_string'])
        #print(x)
        testdata.loc[i,['ref_string_clean1']] = testdata.iloc[i]['ref_string_clean1'].strip().title()
    else:
        testdata.loc[i,['ref_string_clean1']] = testdata.iloc[i]['ref_string'].strip().title()

print(testdata.columns.values)

['Journal_Name' 'Authors_count' 'Authors_list' 'Article_ID' 'content_type'
 'stable_url' 'Ref_type' 'ref_string' 'ref_string_clean1']


In [4]:
#Reading in the master file and correctly formatting the article ID to match back onto the meltef file

#Load in master from Github
#AER_master = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/main/1_Input%20files%20and%20processing%20input%20files/1_1_Input%20files/Master%20lists/Processed/AER_processed.xlsx")

#Local store of AER_master
JPE_master = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/1_Input files and processing input files/1_1_Input files/Master lists/Processed/JPE_processed.xlsx')
for i in range(len(JPE_master)):
    JPE_master.loc[i,['Article_ID']] = JPE_master.loc[i,['stable_url']].to_string().rsplit('/',1)[-1]

JPE_master_condenced  = JPE_master[['Article_ID','year']]
JPE_master_condenced['year'] = JPE_master_condenced['year'].astype('int')
JPE_master_condenced.head()

/var/folders/23/ft9qf9c11rj_vqhbjml910f40000gn/T/ipykernel_16153/679620167.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  JPE_master_condenced['year'] = JPE_master_condenced['year'].astype('int')


,Article_ID,year
0,26549909,2016
1,26549910,2016
2,26549911,2016
3,26549912,2016
4,26549913,2016


In [5]:
# MErging the msterfile onto the new melted file to get additional data
testdata_merged = testdata.merge(JPE_master_condenced, how = 'left', on = 'Article_ID')
testdata_merged = testdata_merged.sort_values(by=['year'])
testdata = testdata_merged.reset_index(drop = True)

In [6]:
# Excel output to get an overview of the data
testdata.head
testdata.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/JPE/Test files/JPE_aff_ref_output_1940_2010_affs_out setup.xlsx') 

In [7]:
list(testdata.columns.values)

['Journal_Name',
 'Authors_count',
 'Authors_list',
 'Article_ID',
 'content_type',
 'stable_url',
 'Ref_type',
 'ref_string',
 'ref_string_clean1',
 'year']

In [8]:
import re
from rapidfuzz import fuzz
import math


#x = 2233

for x in range(len(testdata)):
#for x in range(3627,3628):
#80,114
    authors = testdata.iloc[x]['Authors_list']
    str1 = testdata.iloc[x]['ref_string_clean1']
    #str2 = re.sub('\.','',str1)
#Splitting up the string into authors lines
    ref_split = re.split(r"\n",str1 )
    while("" in ref_split):
        ref_split.remove("")
    #print(ref_split)
    #print(ref_split)

#Finding the authors name and corrosponding affiliations
    count_found = 0
    #Loop through auhor names in author list
    for d in range(len(authors)):
        #Loop through the data that is split into possible authors affiliations
        for j in range(len(ref_split)):
            #Check if the author name is in the split string
            
            #authors[d] = re.sub('\.','',authors[d])

            ratio_author  = fuzz.partial_ratio(ref_split[j], authors[d])
            
            #print(ratio,x)
            if ratio_author > 93:
                count_found = count_found +1
                #print(count_found)

                #append the next line to a column for each author
                # This will ensure each author is accounted for
                if j+1 < len(ref_split):
                    #print(ratio_author)
                    #print(ref_split[j+1], authors[d])
                    testdata.loc[x, str(d)+'_author'] = authors[d]

                    # checking for short erroenous splits
                    if len(ref_split[j+1]) > 3:
                        testdata.loc[x, str(d)+'_author_affs'] = ref_split[j+1]
                    else:
                        testdata.loc[x, str(d)+'_author_affs'] = ''
                    #print(testdata.loc[x, str(d)+'_author_affs'])

            # Prevents double finding of author name, sometimes name occurs twice 
            # but first occurance is the affiliation
            if count_found == len(authors):
                break
                #else:

# Finding the affiliations per author
    #for i in range(0,1):
    #Loop through authors
    for i in range(len(authors)):      
        # Create new affiliations list for each author
        affiliations = []
        #Create the variable or where the authors affilaiton is stored
        item = str(str(i)+'_author_affs')
        #print(item)
        #print(testdata.iloc[x][item])
        # Fuzzy matching loop for possible affiliations

        #Get the affiliation associated with each author
        author_aff = testdata.iloc[x][item]
        print(author_aff)
        if author_aff not in('University Of Texas','University Of California','University','Universi'):
            for z in Affiliation_choices:
                #print(item)
                #print(testdata.iloc[x][item])
                if pd.notnull(author_aff):
                    #print(author_aff)
                    # z is the affilition choice being matched
                    # author_aff is the input string based on authors
                        ratio  = fuzz.partial_ratio(author_aff, z)
                        if ratio > 94:
                            #print(z)
                            #print(item)
                            #print(ratio , x)
                            #print(testdata.iloc[x][str(i)+'_author_affs'])
                            #Appending the affiliation and the ratio to affiliations list
                            affiliations.append([z,ratio])
                                    #Errors
                            if len(affiliations) == 8:
                                break
        elif author_aff == 'University Of Texas':
            affiliations.append(['University Of Texas',100])
        elif author_aff == 'University Of California':
            affiliations.append(['University Of Texas',100])
    
        #Append full affiliations list
        #print(affiliations)
        #testdata.loc[x, '0_affiliations_list'] = affiliations

        #Add each aff to a new column for each author
        for e in range(len(affiliations)):
            testdata.loc[x,str(i)+'_'+ str(e)+'_Auth_aff'] = affiliations[e][0]
            testdata.loc[x,str(i)+'_'+ str(e)+'_Auth_aff_ratio'] = affiliations[e][1]
            print(affiliations[e][0])
            print(affiliations[e][1])
            #print(testdata.loc[x,'Auth_'+str(i)+'_'+ str(e)+'_aff'])
    #print(x)
    

Washington, D.C.
New York University
New York University
100.0
York University
100.0
University Of California, Berkeley
University of California, Berkeley
97.05882352941177
University Of California, Berkeley
University of California, Berkeley
97.05882352941177
New York University
New York University
100.0
York University
100.0
University Of Minnesota.
University Of Minnesota
100.0
Governors University Of Minnesota
97.87234042553192
Ohio State University
Ohio State University
100.0
Stanford University
Stanford
100.0
Stanford University
100.0
Mountain Lakes, New Jersey
Antwerp.
Harvard University
Harvard
100.0
Harvard University
100.0
Washington, D.C.
New York University
New York University
100.0
York University
100.0
New York University
New York University
100.0
York University
100.0
New York University
New York University
100.0
York University
100.0
Iowa State College
Iowa State College
100.0
Colgate University
Colgate University
100.0
Harvard University
Harvard
100.0
Harvard Universit

In [18]:
# Creating a dictionary for consolidation

misspellings = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Misspellings')
comma_in_name = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Comma in name')
colleges = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Colleges')
correct = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Correct')

frames = [misspellings, comma_in_name,colleges,correct]
consolidated_affiliations = pd.concat(frames)
print(consolidated_affiliations.shape)

for i in range(len(consolidated_affiliations)):
    #Replaceing title case and stripping leading space
    #affs.iat[i,0] = affs.iat[i,0].strip().title()
    consolidated_affiliations.iloc[i]['Incorrect spelling'] = consolidated_affiliations.iloc[i]['Incorrect spelling'].strip().title()
    #affs.iat[i,0] = affs.iat[i,0].strip().upper()

consolidated_affiliations_dict = consolidated_affiliations.set_index('Incorrect spelling').to_dict()['Correct spelling']
print(consolidated_affiliations_dict)

(877, 2)
{'Texas A And M College': 'Texas A & M University', 'Texas A And M University': 'Texas A & M University', 'Texas A&M University': 'Texas A & M University', 'Texas Abm University': 'Texas A & M University', 'Texas Ag M University': 'Texas A & M University', 'Texas Agm University': 'Texas A & M University', 'Telaviv University': 'Tel Aviv University', 'Texas A & M University': 'Texas A & M University', 'Texas A. And M. University': 'Texas A & M University', 'Massachusetts Institule Of Technology': 'Massachusetts Institute Of Technology', 'Massachusetts Institute Of Technolog Y': 'Massachusetts Institute Of Technology', 'Yale Unicersity': 'Yale University', 'University Of California, Berkeley': 'University of California, Berkeley', 'University Of California Berkeley': 'UCB', 'University Of California, Davis': 'University of California, Davis', 'University Of California, Irvine': 'University of California, Irvine', 'University Of California, Los Angeles': 'University of California

In [19]:
import numpy as np
testdata = testdata.replace(np.nan, '', regex=True)

for i in range(len(testdata)):
#for i in range(4086,4087):
    #print(testdata.iloc[i]['0_author_affs'])
    testdata.loc[i,['0_author_affs']] =  re.sub(r'\.','',testdata.iloc[i]['0_author_affs'])
    #print(testdata.iloc[i]['0_author_affs'])
    for key in consolidated_affiliations_dict.keys():
        #testdata.loc[i,'0_author_affs_consolidated']  = testdata.iloc[i]['0_author_affs'].replace(key, consolidated_affiliations_dict[key])
        if key in testdata.iloc[i]['0_author_affs']:
            testdata.loc[i,'0_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],testdata.iloc[i]['0_author_affs'])
            print(i)
            print(testdata.loc[i,'0_author_affs_consolidated'])
            #print(re.sub(key, consolidated_affiliations_dict[key],testdata.iloc[i]['0_author_affs']))
        #else:
        #    testdata.loc[i,'0_author_affs_consolidated'] = ''
            #print(testdata.iloc[i]['0_author_affs_consolidated'])
    #testdata.loc[i,['1_author_affs']] =  re.sub(r'\.','',testdata.iloc[i]['1_author_affs'])
    #testdata.loc[i,['2_author_affs']] =  re.sub(r'\.','',testdata.iloc[i]['2_author_affs'])
    auths = re.split(' And ', testdata.iloc[i]['0_author_affs'])

1
New York University
2
University of California, Berkeley
3
New York University
4
University Of Minnesota
5
Ohio State University
6
Stanford University
9
Harvard University
11
New York University
12
New York University
13
Iowa State College
14
Colgate University
15
Harvard University
16
University Of Chicago
19
Ohio State University
20
Iowa State College
21
Duke University
22
University Of North Carolina
23
University Of Michigan
25
University Of Arizona
26
University Of Chicago
28
University of Cambridge, Cambridge
29
Cornell University
30
University Of Edinburgh
31
University Of Minnesota
34
University Of Arkansas
36
Duke University
39
University Of Chicago
40
Duke University
43
Brown University
44
Brown University
45
Iowa State College
51
Brown University
53
Columbia University
54
University Of Chicago And Cowles Commission
55
University Of Chicago
56
Columbia University
62
University Of Chicago
63
Princeton University
64
University Of Michigan
67
University Of Chicago
68
Northwest

In [11]:
# Testing how the consolidation works

#Universities_dict = {("UN","University of California, San Diego"):"UCSD", "Uniersity of Illinois": "University of Illinois"}
Universities_dict = {"University of California, San Diego":"UCSD","University of Catifornia, San Diego":"UCSD", "Uniersity of Illinois": "University of Illinois"}

aff_string = "University of California, San Diego and Uniersity of Illinois and University of Catifornia, San Diego"
aff_string = "University Of California, Berkeley, And The Ford Foundation, Santiago, Chile"
#
for key in consolidated_affiliations_dict.keys():
    aff_string = aff_string.replace(key, consolidated_affiliations_dict[key])
    #print(consolidated_affiliations_dict[key])

print(aff_string)

UCB, And The Ford Foundation, Santiago, Chile


In [20]:
print(testdata.columns.values)

['Journal_Name' 'Authors_count' 'Authors_list' 'Article_ID' 'content_type'
 'stable_url' 'Ref_type' 'ref_string' 'ref_string_clean1' 'year'
 '0_author_affs' '0_author_affs_consolidated' '1_author_affs'
 '2_author_affs' '3_author_affs' '4_author_affs' '5_author_affs']


In [21]:
testdata = testdata[['Journal_Name' ,'Authors_count', 'Authors_list', 'Article_ID', 'content_type',
 'stable_url', 'Ref_type' ,'ref_string', 'ref_string_clean1', 'year',
 '0_author_affs', 
 '0_author_affs_consolidated',
 '1_author_affs', 
 '2_author_affs', 
 '3_author_affs' ,
 '4_author_affs',
 '5_author_affs',]]

In [22]:
# Excel output to get an overview of the data
testdata.head
testdata.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/JPE/Test files/JPE_aff_ref_output_1940_2010_affs_out authors.xlsx') 